## Data Loading
เตรียม Dataset โดยแบ่งเป็น 8 files แล้วค่อย concat เป็น DataFrame เดียว เพื่อไม่ให้ memory

In [37]:
import pandas as pd
import glob

# 1. สร้าง list ของไฟล์ทั้งหมด (ใช้ glob)
file_list = sorted(glob.glob("./dataset/bangkok_traffy_part_*.csv"))

# 2. อ่านทีละไฟล์แล้ว append
df_list = []
for file in file_list:
    print(f"Reading {file} ...")
    df_list.append(pd.read_csv(file))

# 3. รวมทั้งหมดเป็น DataFrame เดียว
df = pd.concat(df_list, ignore_index=True)

# 4. ตรวจสอบจำนวนแถว
print(f"Total rows: {len(df)}")

Reading ./dataset\bangkok_traffy_part_1.csv ...
Reading ./dataset\bangkok_traffy_part_2.csv ...
Reading ./dataset\bangkok_traffy_part_3.csv ...
Reading ./dataset\bangkok_traffy_part_4.csv ...
Reading ./dataset\bangkok_traffy_part_5.csv ...
Reading ./dataset\bangkok_traffy_part_6.csv ...
Reading ./dataset\bangkok_traffy_part_7.csv ...
Reading ./dataset\bangkok_traffy_part_8.csv ...
Total rows: 787026


# เช็คค่าต่างๆจาก DataFrame

**เช็ค จำนวน records**

In [29]:
print(len(df))

787026


**เช็ค province ที่มีทั้งหมด**

In [25]:
print(df['province'].unique())

['กรุงเทพมหานคร' 'ศรีสะเกษ' 'นนทบุรี' 'สมุทรปราการ' 'จังหวัดกรุงเทพมหานคร'
 'สมุทรสาคร' 'ปทุมธานี' 'นครปฐม' 'สุพรรณบุรี' 'เชียงราย' nan 'จังหวัดLac'
 'จังหวัดนครปฐม' 'จังหวัดSahel' 'จังหวัดสมุทรสาคร' 'จังหวัดชลบุรี'
 'จังหวัดBorno' 'จังหวัดTillabéri' 'จังหวัดจังหวัด กรุงเทพมหานคร'
 'จังหวัดสมุทรปราการ' 'จังหวัดปทุมธานี' 'จังหวัดนนทบุรี'
 'จังหวัดพระนครศรีอยุธยา' 'จังหวัดนครศรีธรรมราช' 'จังหวัดฉะเชิงเทรา'
 'จังหวัดชุมพร' 'จังหวัดPerak' 'จังหวัดRiau' 'ชลบุรี' 'จังหวัดBangkok'
 'จังหวัดจังหวัดกรุงเทพมหานคร' 'จังหวัดนครสวรรค์' 'จังหวัดสงขลา' 'ลำพูน'
 'จังหวัดจ.ฉะเชิงเทรา' 'พิษณุโลก' 'ภูเก็ต' 'นครราชสีมา' 'จังหวัดสระบุรี'
 'ราชบุรี' 'สงขลา' 'จังหวัดประจวบคีรีขันธ์' 'สุราษฎร์ธานี' 'จังหวัดลพบุรี'
 'เชียงใหม่' 'ยโสธร' 'กาญจนบุรี' 'พังงา' 'อุดรธานี' 'จังหวัดราชบุรี'
 'จังหวัดเพชรบุรี' 'สุรินทร์' 'นครสวรรค์' 'ลพบุรี' 'จังหวัดจังหวัดพะเยา'
 'จังหวัดเชียงใหม่' 'จังหวัดจังหวัด' 'บุรีรัมย์' 'พระนครศรีอยุธยา'
 'จังหวัดจังหวัดพระนครศรีอยุธยา' 'จังหวัดกรุงเทพฯ' 'แพร่' 'ขอนแก่น'
 'จังหวัดจ.สุพรรณบุรี' 

**เช็ค language encoding ของ province**

In [23]:
import chardet

with open('./dataset/bangkok_traffy_100.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))
print(result)


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


# Data Cleaning & Quality Assurance

**เคลียร์ : ข้อมูลที่ไม่ได้อยู่ในกรุงเทพ**

In [49]:
import pandas as pd

# สมมติ df['province'] มีค่าเหมือนคุณ
# ให้ normalize ก่อน: lowercase + strip space
df['province_clean'] = df['province'].astype(str).str.lower().str.strip()

# list ของ pattern ที่ต้อง match (ไม่ต้องแม่นยำทั้งหมด)
bangkok_patterns = ["bangkok", "กรุงเทพ"]  # รวมภาษาไทย + อังกฤษ

# filter row ที่ตรงกับ pattern
df_bangkok = df[df['province_clean'].str.contains('|'.join(bangkok_patterns), na=False)].copy()

print(f"จำนวนแถว Bangkok: {len(df_bangkok)}") # จาก 787,026 เหลือ 785,662 หายไป "1364" records
print(df_bangkok['province'].unique())

# ถ้าไม่ต้องการ column ชั่วคราว
df_bangkok.drop(columns=['province_clean'], inplace=True)


จำนวนแถว Bangkok: 785662
['กรุงเทพมหานคร' 'จังหวัดกรุงเทพมหานคร' 'จังหวัดจังหวัด กรุงเทพมหานคร'
 'จังหวัดBangkok' 'จังหวัดจังหวัดกรุงเทพมหานคร' 'จังหวัดกรุงเทพฯ']


**เคลียร์ column ที่ไม่ได้ใช้ (After Clean)**

In [48]:
# 1. ระบุชื่อ column ที่ต้องการลบ
columns_to_drop = ['ticket_id', 'comment', 'photo', 'photo_after', 'coords', 'address', 'province', 'count_reopen']  # แก้เป็นชื่อ column ที่ไม่ต้องการ

# 2. ลบ column เหล่านั้น
df_clean = df.drop(columns=columns_to_drop)

# 3. ตรวจสอบผลลัพธ์
print(df_clean.head())
print(df_clean.columns)

# 4. บันทึกไฟล์ใหม่ (ถ้าต้องการ)
# df_clean.to_csv("dataset/bangkok_traffy_clean.csv", index=False)


                  type                      organization subdistrict  \
0          {ความสะอาด}                        เขตบางซื่อ         NaN   
1  {น้ำท่วม,ร้องเรียน}      เขตประเวศ,ฝ่ายโยธา เขตประเวศ     หนองบอน   
2              {สะพาน}                           เขตสาทร     ยานนาวา   
3            {น้ำท่วม}    เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ         NaN   
4        {น้ำท่วม,ถนน}  เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว    ลาดพร้าว   

   district                      timestamp      state  star  \
0       NaN  2021-09-03 12:51:09.453003+00  เสร็จสิ้น   NaN   
1    ประเวศ  2021-09-19 14:56:08.924992+00  เสร็จสิ้น   4.0   
2      สาทร  2021-09-26 05:03:52.594898+00  เสร็จสิ้น   NaN   
3       NaN  2021-10-14 10:45:27.713884+00  เสร็จสิ้น   NaN   
4  ลาดพร้าว  2021-12-09 12:29:08.408763+00  เสร็จสิ้น   5.0   

                   last_activity province_clean  
0  2022-06-04 15:34:14.609206+00  กรุงเทพมหานคร  
1  2022-06-21 08:21:09.532782+00  กรุงเทพมหานคร  
2  2022-06-06 01:17:12.272904+00  กรุง

**เคลียร์ และ จัดโครงสร้าง Timestamp and Last Actitity ใหม่**

In [42]:
for col in ["timestamp", "last_activity"]:
    # แปลงเป็น datetime, UTC โดยให้ pandas infer format เอง
    df_clean[col] = pd.to_datetime(df_clean[col], utc=True, errors="coerce")
    
    # แปลง timezone เป็น Asia/Bangkok
    df_clean[col] = df_clean[col].dt.tz_convert("Asia/Bangkok")
    
    # เอา microseconds และวินาทีออก → เหลือ Year-Month-Day Hour:Minute
    df_clean[col] = df_clean[col].dt.strftime("%Y-%m-%d %H:%M")

# บันทึก CSV ใหม่
df_clean.to_csv("./dataset/bangkok_traffy_clean.csv", index=False)

print("Cleaned CSV saved with timestamp and last_activity formatted.")


Cleaned CSV saved with timestamp and last_activity formatted.
